In [1]:
# Import relevant packages
import duckdb, pandas as pd

# Define Variables

In [2]:
dic_leagues = {
    "England": 39,     # Premier League
    "France": 61,      # Ligue 1
    "Germany": 78,     # Bundesliga
    "Italy": 135,      # Serie A
    "Spain": 140,      # La Liga
}

# Define Functions

In [3]:
db_path = r"C:/Users/campo/Desktop/sports betting/warehouse.duckdb"
con = duckdb.connect(db_path, read_only=False) 

def get_table(table_name):
    query = f"""select * from {table_name} order by 1"""
    df_matches = con.execute(query).fetchdf()
    return df_matches

-----

# Gather tables

In [4]:
df_odds = get_table('warehouse.main.odds')
df_inj = get_table('warehouse.main.fact_injuries')
df_fixt = get_table('warehouse.main.fact_fixtures')
df_players = get_table('warehouse.main.fact_player_stats_match')

### Right now, we only have german odds for 2025. Let's restrict the other tables.
df_inj = df_inj[(df_inj.league_id == dic_leagues['Germany']) & (df_inj.season == 2025)]
df_fixt = df_fixt[(df_fixt.league_id == dic_leagues['Germany']) & (df_fixt.season == 2025)]
df_players = df_players[(df_players.league_id == dic_leagues['Germany']) & (df_players.season == 2025)]

##### Injury table

In [5]:
df_inj.head(5)

,league_id,season,team_id,player_id,player_name,status,reason,start_date,expected_return,fixture_id,updated_ts
14139,78,2025,186,474789,Abdoulie Ceesay,None,None,NaT,NaT,1388320,2025-10-26 17:04:01.879562
14140,78,2025,170,279993,A. Kade,None,None,NaT,NaT,1388335,2025-10-26 17:04:01.880558
14141,78,2025,175,48475,S. Hefti,None,None,NaT,NaT,1388339,2025-10-26 17:04:01.880558
14142,78,2025,162,973,M. Weiser,None,None,NaT,NaT,1388343,2025-10-26 17:04:01.880558
14143,78,2025,173,1144,L. Klostermann,None,None,NaT,NaT,1388341,2025-10-26 17:04:01.880558


##### Fixtures table

In [6]:
df_fixt.head(5)

,fixture_id,league_id,season,round,date_utc,venue_id,venue_name,status_short,home_team_id,away_team_id,home_goals,away_goals,referee,updated_ts
4578,1388308,78,2025,Regular Season - 1,2025-08-22 14:30:00,<NA>,Allianz Arena,FT,157,173,6,0,"Florian Badstubner, Germany",2025-10-26 17:03:39.176120
4579,1388309,78,2025,Regular Season - 1,2025-08-23 09:30:00,699,BayArena,FT,168,167,1,2,"Daniel Siebert, Germany",2025-10-26 17:03:39.178193
4580,1388310,78,2025,Regular Season - 1,2025-08-24 11:30:00,20471,Borussia-Park,FT,163,175,0,0,"Sascha Stegemann, Germany",2025-10-26 17:03:39.185027
4581,1388311,78,2025,Regular Season - 1,2025-08-23 09:30:00,<NA>,Deutsche Bank Park,FT,169,162,4,1,"Harm Osmers, Germany",2025-10-26 17:03:39.178193
4582,1388312,78,2025,Regular Season - 1,2025-08-23 09:30:00,12717,Europa-Park Stadion,FT,160,170,1,3,"Felix Zwayer, Germany",2025-10-26 17:03:39.177132


##### Players stats table

In [7]:
display(df_players.head(5))
print(f"Columns = {df_players.columns}")

,fixture_id,league_id,season,team_id,player_id,player_name,team_name,position,number,is_captain,...,duels_total,duels_won,dribbles_attempts,dribbles_success,fouls_committed,fouls_drawn,yellow,red,offsides,updated_ts
159050,1388308,78,2025,173,513776,Yan Diomande,RB Leipzig,F,49,False,...,7,2,2,2,1,<NA>,0,0,<NA>,2025-10-26 17:03:53.900437
159051,1388308,78,2025,157,125171,Josip Stanišić,Bayern München,D,44,False,...,4,3,<NA>,<NA>,<NA>,1,0,0,<NA>,2025-10-26 17:03:53.899414
159052,1388308,78,2025,173,25158,David Raum,RB Leipzig,D,22,True,...,10,5,2,1,2,2,0,0,<NA>,2025-10-26 17:03:53.899414
159053,1388308,78,2025,173,86,Loïs Openda,RB Leipzig,F,11,False,...,3,2,1,<NA>,<NA>,2,0,0,<NA>,2025-10-26 17:03:53.900437
159054,1388308,78,2025,157,184,Harry Kane,Bayern München,F,9,False,...,8,6,3,2,<NA>,4,0,0,1,2025-10-26 17:03:53.899414


Columns = Index(['fixture_id', 'league_id', 'season', 'team_id', 'player_id',
       'player_name', 'team_name', 'position', 'number', 'is_captain',
       'minutes', 'rating', 'shots_total', 'shots_on', 'goals', 'assists',
       'saves', 'passes_total', 'passes_key', 'passes_accuracy', 'tackles',
       'interceptions', 'blocks', 'duels_total', 'duels_won',
       'dribbles_attempts', 'dribbles_success', 'fouls_committed',
       'fouls_drawn', 'yellow', 'red', 'offsides', 'updated_ts'],
      dtype='object')


##### Odds table

In [8]:
df_odds.head(5)

,event_id,sport_key,sport_title,league_key,market,bookmaker,last_update,commence_time,home_team,away_team,price_home,price_draw,price_away,join_key,season
0,15c8645215dcf291ab411c222a965c29,soccer_germany_bundesliga,Bundesliga - Germany,soccer_germany_bundesliga,h2h,Winamax (FR),2025-10-26 12:04:07-04:00,2025-11-08 12:30:00-05:00,Borussia Monchengladbach,1. FC Köln,2.00,3.35,2.65,2025-11-08|Borussia Monchengladbach|1. FC Köln,2025
1,15c8645215dcf291ab411c222a965c29,soccer_germany_bundesliga,Bundesliga - Germany,soccer_germany_bundesliga,h2h,Winamax (DE),2025-10-26 12:03:42-04:00,2025-11-08 12:30:00-05:00,Borussia Monchengladbach,1. FC Köln,2.15,3.75,2.95,2025-11-08|Borussia Monchengladbach|1. FC Köln,2025
2,15c8645215dcf291ab411c222a965c29,soccer_germany_bundesliga,Bundesliga - Germany,soccer_germany_bundesliga,h2h,Parions Sport (FR),2025-10-26 12:04:21-04:00,2025-11-08 12:30:00-05:00,Borussia Monchengladbach,1. FC Köln,1.93,3.45,2.75,2025-11-08|Borussia Monchengladbach|1. FC Köln,2025
3,15c8645215dcf291ab411c222a965c29,soccer_germany_bundesliga,Bundesliga - Germany,soccer_germany_bundesliga,h2h,Coolbet,2025-10-26 12:03:16-04:00,2025-11-08 12:30:00-05:00,Borussia Monchengladbach,1. FC Köln,2.25,3.75,3.00,2025-11-08|Borussia Monchengladbach|1. FC Köln,2025
4,15c8645215dcf291ab411c222a965c29,soccer_germany_bundesliga,Bundesliga - Germany,soccer_germany_bundesliga,h2h,1xBet,2025-10-26 12:04:06-04:00,2025-11-08 12:30:00-05:00,Borussia Monchengladbach,1. FC Köln,2.29,3.90,3.10,2025-11-08|Borussia Monchengladbach|1. FC Köln,2025


# Merge data

##### The challenge is we have two different providers (TheOdds and API-Football). 
##### Ugly, but as a first step we will use company names to map both dataframes.

In [ ]:
# Football data
df = pd.merge(df_inj, df_fixt, on=['fixture_id', 'league_id', 'season'], how='inner', suffixes=['_injuries', '_fixtures'])
df = pd.merge(df, df_players, on=['league_id', 'season', 'team_id', 'fixture_id'], how='inner', suffixes=['_injuries', '_players'])

In [70]:
df_companies_apifootball = df[['team_name', 'team_id']].drop_duplicates().set_index('team_id')['team_name'].copy()
df_companies_odds = df_odds['home_team'].drop_duplicates().copy()

In [ ]:
already_mapped = [comp for comp in df_companies_apifootball.to_list() if comp in df_companies_odds.to_list()]
in_api_not_in_odds = [comp for comp in df_companies_apifootball.to_list() if comp not in df_companies_odds.to_list()]
in_odds_not_in_api = [comp for comp in df_companies_odds.to_list() if comp not in df_companies_apifootball.to_list()]

print(f"already mapped: {already_mapped}")
print(f"in_api_not_in_odds: {in_api_not_in_odds}") # also contains team that got relegated.
print(f"in_odds_not_in_api: {in_odds_not_in_api}")

already mapped: ['FC St. Pauli', 'Werder Bremen', 'RB Leipzig', 'Bayer Leverkusen', 'VfB Stuttgart', 'FSV Mainz 05', '1. FC Heidenheim', 'Borussia Dortmund', 'VfL Wolfsburg', 'Eintracht Frankfurt', 'Union Berlin', 'SC Freiburg']
in_api_not_in_odds: ['FC Augsburg', 'Hamburger SV', 'Borussia Mönchengladbach', '1.FC Köln', 'Bayern München', '1899 Hoffenheim']
in_odds_not_in_api: ['Borussia Monchengladbach', '1. FC Köln', 'Augsburg', 'Bayern Munich', 'FC Bayern München']


### mapping is annoying ... let's give API-Football another shot for odds.

In [123]:
from dotenv import load_dotenv
load_dotenv()
import os, requests, pandas as pd

API_KEY = os.getenv("API_FOOTBALL_KEY")
BASE = "https://v3.football.api-sports.io"
HEADERS = {"x-apisports-key": API_KEY}

resp = requests.get(f"{BASE}/odds", headers=HEADERS, timeout=30, params={'league':39, 'season':2025, "page":4})

j = resp.json()  # dict with keys: get, parameters, errors, results, response(list)
data = j.get("response", [])
# Flatten the common fields into a tidy DataFrame
df_odds = pd.json_normalize(data, sep="_")
display(df_odds)
## too weak. looks like we will need to go with TheOdds. we can't get more than the last game day.

""


## Conclusion: 

we will go with TheOdds for quotes and map to football data using team names and fixture date. it is not a big issue, because we are dealing with a group of 20 teams. mapping errors should not be an issue, as long as the names are consistent. looks like, at least in 2025, they are.